In [1]:
!cd

E:\Projets\Dog and cat


In [2]:
!kaggle competitions download -c dogs-vs-cats -p Data/

dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import os
import zipfile
import matplotlib.image as image
from io import BytesIO
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from keras.utils import to_categorical
from keras.layers import Conv2D,MaxPooling2D,Activation,Dense,Flatten,Dropout
from keras.models import Sequential
from keras.initializers import RandomNormal,RandomUniform
from keras.optimizers import Adam,RMSprop
from sklearn.model_selection import train_test_split

In [4]:
print('Extracting files ....')
with zipfile.ZipFile('Data/dogs-vs-cats.zip') as zfiles:
    for file in zfiles.namelist():
        try:
            zfiledata = BytesIO(zfiles.read(file))
            efile = zipfile.ZipFile(zfiledata)
            efile.extractall('data/')
        except:
            print('An error occured')
print('Extraction completed ....')
print('Extracted file location (data/).... ')

Extracting files ....
1
An error occured
1
1
Extraction completed ....
Extracted file location (data/).... 


In [4]:
train_data_path = 'Data/train/'
test_data_path = 'Data/test1/'

In [5]:
def get_data_ready(path):
    print('Loading Files .... ')
    print('Please Wait ....')
    label, ids, data = [],[],[]
    for files in os.walk(path):
        for file in files[2]:
            filename, extension = os.path.splitext(file)
            if extension == '.jpg':
                load_path = path+file
                img = Image.open(load_path)
                img = img.resize((100,100),resample=Image.NEAREST)
                data.append(np.asarray(img))
                try:          
                    ids.append(filename.split('.')[1])
                    if filename.split('.')[0] == 'cat':
                        label.append(1)
                    else:
                        label.append(0)
                except:
                    ids.append(filename.split('.'))
            else:
                print('Not a .jpg file  the actual extension is {}'.format(extension))
                
    data = np.asarray(data)
    label = np.asarray(label)
    print('Files Loading Complete ....')
    return label,ids,data

In [6]:
train_labels,train_ids,train_data = get_data_ready(train_data_path)

Loading Files .... 
Please Wait ....
Files Loading Complete ....


In [7]:
test_labels,test_ids,test_data = get_data_ready(test_data_path)

Loading Files .... 
Please Wait ....
Files Loading Complete ....


In [8]:
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
train_data = train_data/255
test_data = test_data/255

In [9]:
train_data.shape , test_data.shape

((25000, 100, 100, 3), (12500, 100, 100, 3))

In [10]:
model1 = Sequential()

model1.add(Conv2D(filters = 32 ,
                  kernel_size = (3,3),
                  strides = (2,2),
                  padding = 'same',
                  activation='relu',
                  input_shape = train_data.shape[1:]))

model1.add(MaxPooling2D(pool_size=(4, 4),
                        strides=(1, 1),
                        padding='valid'))

model1.add(Dropout(0.25))

model1.add(Conv2D(filters = 64 ,
                  kernel_size = (3,3),
                  strides = (1,1),
                  padding = 'valid',
                  activation='relu'))

model1.add(MaxPooling2D(pool_size=(4, 4),
                        strides=(1, 1), 
                        padding='valid'))

model1.add(Conv2D(filters = 64 ,
                  kernel_size = (3,3),
                  strides = (2,2),
                  padding = 'same',
                  activation='relu'))
model1.add(MaxPooling2D(pool_size=(4, 4),
                        strides=(1, 1),
                        padding='valid'))

model1.add(Dropout(0.25))
model1.add(Conv2D(filters = 64 ,
                  kernel_size = (3,3),
                  strides = (1,1),
                  padding = 'valid',
                  activation='relu'))

model1.add(MaxPooling2D(pool_size=(4, 4),
                        strides=(1, 1),
                        padding='valid'))


model1.add(Conv2D(filters = 128 ,
                  kernel_size = (3,3),
                  strides = (2,2),
                  padding = 'same',
                  activation='relu'))

model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(units=64,activation = 'relu'))
model1.add(Dense(units=128,activation = 'relu'))
model1.add(Dense(units=256,activation = 'relu'))
model1.add(Dense(units=1,activation = 'sigmoid'))


model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 50, 50, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 47, 47, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 45, 45, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 42, 42, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 64)        0

In [11]:
opt = RMSprop(learning_rate= 0.1)
model1.compile(optimizer=opt,
               loss = 'binary_crossentropy',
               metrics = 'accuracy')

In [12]:
x_train,x_valid,y_train,y_valid = train_test_split(train_data,train_labels,
                                                   random_state=42,                            
                                                   test_size=0.2)

In [13]:
model1_summary = model1.fit(x_train,y_train,
                            batch_size=32,
                            shuffle=True,
                            epochs = 10,
                            validation_data=(x_valid,y_valid))

Epoch 1/10
625/625 [==============================] - 207s 331ms/step - loss: 199237520.0000 - accuracy: 0.5003 - val_loss: 0.6976 - val_accuracy: 0.4970
Epoch 2/10
625/625 [==============================] - 195s 311ms/step - loss: 310933.0000 - accuracy: 0.5039 - val_loss: 0.7046 - val_accuracy: 0.5030
Epoch 3/10
625/625 [==============================] - 192s 308ms/step - loss: 0.6963 - accuracy: 0.4985 - val_loss: 0.6933 - val_accuracy: 0.4970
Epoch 4/10
625/625 [==============================] - 192s 307ms/step - loss: 0.6953 - accuracy: 0.5021 - val_loss: 0.7011 - val_accuracy: 0.5030
Epoch 5/10
625/625 [==============================] - 194s 310ms/step - loss: 0.6956 - accuracy: 0.4931 - val_loss: 0.6941 - val_accuracy: 0.4970
Epoch 6/10
625/625 [==============================] - 196s 314ms/step - loss: 0.6955 - accuracy: 0.5009 - val_loss: 0.6954 - val_accuracy: 0.4970
Epoch 7/10
625/625 [==============================] - 219s 350ms/step - loss: 0.6956 - accuracy: 0.4983 - val_l

In [14]:
type(y_train)

numpy.ndarray